In [ ]:
import docx
import os
from document_formatter_config import DocumentFormatterConfig
from document_formatting_agent import DocumentFormattingAgent
from config import INPUT_DOCX, INPUT_DIR, STYLE_CONFIG_FILENAME, STYLE_SCHEMA_FILENAME, OUTPUT_DOCX

In [3]:
doc = docx.Document(INPUT_DOCX)

In [4]:
formatter_config = DocumentFormatterConfig.load_and_validate_yaml(
    input_dir=INPUT_DIR,
    style_filename=STYLE_CONFIG_FILENAME,
    schema_filename=STYLE_SCHEMA_FILENAME
)

In [5]:
agent = DocumentFormattingAgent(doc, formatter_config)
agent.apply_all_styles()

In [6]:
doc.save(OUTPUT_DOCX)